In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [24]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import timedelta

# Reflect Tables into SQLAlchemy ORM

In [25]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import inspect
from sqlalchemy import create_engine, func

In [26]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [27]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect = True)

In [28]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [29]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [30]:
# Create our session (link) from Python to the DB
session = Session(engine)
inspector = inspect(engine)

# Exploratory Precipitation Analysis

In [31]:
#Explore the database
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [32]:
# Find the most recent date in the data set.
recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
recent_date

('2017-08-23')

In [33]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.

# Perform a query to retrieve the data and precipitation scores
precipitation = session.query(Measurement.date, Measurement.prcp).\
                                    filter(func.strftime("%Y-%m-%d", Measurement.date) >= recent_date).\
                                    filter(Measurement.date).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
precipitation_df = pd.DataFrame(precipitation).set_index("date")

# Sort the dataframe by date
precipitation_df = precipitation_df.sort_values(by='Date', ascending=True)

# Use Pandas Plotting with Matplotlib to plot the data
plt.figure(figsize=(20,10));
plt.bar(precipitation_df["Date"], precipitation_df["Precipitation"], align='center', width=2);

plt.xlabel('Date')
plt.ylabel('Precipitation (inches)')
plt.title('Precipitation by Date')

plt.grid(b=bool, which='major', axis='y', color='grey', alpha=0.55);
plt.xticks("", rotation=90, fontweight="bold", aplha=0.55);
plt.yticks(rotation=0, fontweight="bold");

plt.savefig("Output/prcp_fig.png")

plt.show()


InterfaceError: (sqlite3.InterfaceError) Error binding parameter 1 - probably unsupported type.
[SQL: SELECT measurement.date AS measurement_date, measurement.prcp AS measurement_prcp 
FROM measurement 
WHERE strftime(?, measurement.date) >= ? AND measurement.date]
[parameters: ('%Y-%m-%d', ('2017-08-23',))]
(Background on this error at: http://sqlalche.me/e/rvf5)

In [34]:
# Use Pandas to calculate the summary statistics for the precipitation data
precipitation_df.describe()

NameError: name 'precipitation_df' is not defined

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
session.query(func.count(Station.id)).all()

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
session.query(Station.station, func.count(Station.id)).\
    filter(Station.station == Measurement.station).\
    group_by(Measurement.station).\
    order_by(func.count(Station.station).desc()).all()

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
session.query(Measurement.station, func.min(Measurement.tobs),
             func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
    filter(Measurement.station == "USC00519281")
    group_by(Measurement.station).all()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temperature = session.query(Measurement.date, Measurement.tobs).\
    filter(Measurement.station == "USC00519281").\
    filter(func.strftime("%Y-%m-%d", Measurement.date) >= recent_date).all()

temperature_df = pd.DataFrame(temperature, columns=['Date', 'Temperature'])
temperature_df = temperature_df.sort_values(by='Date', ascending=True)
temperature_df.set_index('Date', inplace=True)

#Plot figure as histogram
plt.figure(figsize=[20,10])
n, bins, patches = plt.hist(x=temperature_df["Temperature"], bins=12)

plt.grid(axis='y')
plt.xlabel
plt.ylabel
plt.xticks
plt.yticks
plt.title('Temperature Frequency Histogram for Station: USC00519281')

plt.savefig('Output/temp_histogram.png')

plt.show()

# Close session

In [ ]:
# Close Session
session.close()